In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
import time

# Initialize pipeline
pipeline = rs.pipeline()

# Create a config object
config = rs.config()

# Enable color and depth streams
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

profile = pipeline.get_active_profile()
depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.depth))
depth_intrinsics = depth_profile.get_intrinsics()
print("Depth Intrinsics: ", depth_intrinsics)

# Create an align object
# This will align the depth frame to the color frame
align_to = rs.stream.color
align = rs.align(align_to)

# Start timer
start_time = time.time()

saved = False

try:
    while True:
        # Wait for frames from the camera
        frames = pipeline.wait_for_frames()

        # Align the depth frame to the color frame
        aligned_frames = align.process(frames)

        # Get aligned depth and color frames
        aligned_depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()

        if not aligned_depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        aligned_depth_image = np.asanyarray(aligned_depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Apply colormap to aligned depth image for better visualization
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(aligned_depth_image, alpha=0.03), cv2.COLORMAP_JET)

        images = np.zeros((480,640,4), dtype=np.uint16)
        images[:, :, :3] = color_image
        images[:, :, 3] = aligned_depth_image

        # Show images
        cv2.imshow('RealSense - Aligned Color and Depth', color_image)

        # Save aligned frames after 5 seconds
        if not saved and time.time() - start_time >= 10:
            np.save('aligned_images.npy', images)
            print("Aligned images saved as numpy files.")
            saved = True
            break

        # Break the loop on key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Stop streaming
    pipeline.stop()

# Clean up
cv2.destroyAllWindows()


Depth Intrinsics:  [ 640x480  p[314.029 237.872]  f[386.867 386.867]  Brown Conrady [0 0 0 0 0] ]
Aligned images saved as numpy files.


In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
import time
imagesss = np.load('aligned_images.npy')
print(imagesss.shape)
print(imagesss[0:5,0:5,0])

color = imagesss[:, :, :3]/255
depth = imagesss[:, :, 3]
print(color.shape)

cv2.imshow('RealSense - Aligned Color and Depth', depth)
cv2.waitKey(0)
cv2.destroyAllWindows()
